In [1]:
import re, json, pathlib, logging, time, argparse, pprint 
import datetime as dt
import importlib.resources as rsrc
from itertools import chain
from operator import itemgetter 
#
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import nltk as nltk
from nltk.corpus import stopwords
from whoosh.lang.porter import stem
#from googletrans import Translator
from pygoogletranslation import Translator
import circlify as crcf
import matplotlib.pyplot as plt
import random

import polmap.polmap as plmp
import polmap.clean_docs as cldc
import postprocess.postprocess as pspr


##MM imports
#from polmap.polmap import make_directories, preprocess_text, doc2text, SDGrefs_mapper # replaced the keyword processing block

#import fromRtoPython

from docx2python import docx2python
 
#from polmap.polmap import make_directories, preprocess_text, doc2text # replaced the keyword processing block

In [5]:
#stopwordset(stopwords.words('english'))
stop_words = set(stopwords.words('english'))-set(['no','not','nor'])
#stop_words.remove('all')

#print(stop_words)

test_keys=test_keys=['innovative infrastructure', 'innovative industry', 'indsutrial innovation', 'industry technology', 'research and development', 'R&D', 'innovative technology', 'green technology', 'Expenditure in Research', 'Expenditure in Innovation', 'personnel employed in RD&I', 'foster innovation', 'Investing in digital infrastructure', 'research and innovation', 'innovation and research', 'Horizon Europe', 'Eurostars', 'EUREKA', 'establish research cells', 'foster research and partnerships', 'set up research partnerships', 'networks for research', 'facilitate innovation', 'foster technological progress', 'foster research cooperation', 'facilitate research cooperation', 'improve research cooperation', 'strengthen research cooperation']

for key in test_keys:
    print(plmp.preprocess_text(key, stop_words, verbose=False))

 innov infrastructur 
 innov . 
 industri 
 indsutri 
 innov 
 industri technolog 
 research develop 
 r&d 
 innov technolog 
 green technolog 
 expenditur research 
 expenditur innov 
 personnel emploi rd&i 
 foster innov 
 invest digit infrastructur 
 research innov 
 innov research 
 horizon europ 
 eurostar 
 eureka 
 establish research cell 
 foster research partnership 
 set research partnership 
 network research 
 facilit innov 
 foster technolog progress 
 foster research cooper 
 facilit research cooper 
 improv research cooper 
 strengthen research cooper 


In [ ]:
a='foo'
print(f'{a}'+'\nbar')

In [ ]:
input_dir=pathlib.Path('output/DirD_/output/2-doc_text')
files = sorted(input_dir.glob('*.txt'))
file_df=pd.DataFrame(files,columns=['Path'])
file_df['File']=file_df['Path'].apply(lambda x: x.name)
file_df['PRJ']=file_df['File'].str.extract(r'_PRJ_(\d{5})')
file_df['PP']=file_df['File'].str.extract(r'_PP_(\d{5})')

file_df.head()

In [ ]:
doc_text={}
for file_ in files:
    with open(file_, 'r') as f:    
        doc_text[file_.name]=f.read()
    
    cleaned_text, cleaned_dict=cldc.clean_jpb(doc_text[file_.name])#[0]

    if all(cleaned_dict.values()):
        #print(f'{file_.name}: Ok\n\n')
        pass
    else:
        print('\n\n',file_.name,'\n\n')
        for key, value in cleaned_dict.items():        
            if not value:
                print(f'{key}:  {bool(value)}\n\n\n')
          
    
    destpath=file_.parent/'cleaned'/f'{file_.name}_cleaned.txt'.replace('docx.txt_','docx_')
    #print(destpath)
    destpath.parent.mkdir(mode=0o777, parents=True, exist_ok=True)
    with destpath.open('w') as f_:
         f_.write(cleaned_text)

    #print(destpath,'\n'*2, destpath.exists())

    destpath= destpath.parent/ destpath.name.replace('txt', f'_{all(cleaned_dict.values())}_.json')
    with open(destpath, "w") as fjson:
        json.dump(cleaned_dict, fjson, indent = 6)
    #doc_text[file_.name]=re.findall(r"SDG\n\n([a-zA-z, -]{1,})\n", doc_text[file_.name], flags=re.IGNORECASE)

In [ ]:
res_path = pathlib.Path('./output/DirD_cleaned_/output/6-results/results_cleaned.xlsx')

filename=res_path.parent / 'DirD_cleaned_'

mock_target_df = pd.read_excel(res_path,sheet_name='target_dat') #pd.DataFrame({
#     'Name': target_list,
#     'Value': [np.random.randint(0, 100) for i in target_list]
# })
mock_target_df=mock_target_df[['Target','Count']]

mock_target_df.rename(columns={"Target": "Name", "Count": "Value"}, inplace=True)
mock_target_df=mock_target_df.groupby(by=['Name']).sum().
mock_target_df =  mock_target_df[mock_target_df['Value'] != 0]

mock_goals = pd.read_excel(res_path,sheet_name='goal_overview')
mock_goals = mock_goals['Count']

In [ ]:
mock_target_df.reset_index().head()

In [ ]:
mock_target_df['Value'].tolist()

In [ ]:
mock_target_df['Name'].tolist()

In [ ]:
doc_text[0]

In [ ]:
#for name, value in doc_text.items():
    #print(name, value)

file_df['SDG']=file_df['File'].apply(lambda x: doc_text[x][0])
file_df.head()

In [ ]:
dird_df=pd.read_excel('input/Projects_overview.xlsx')
dird_df.head()

In [ ]:
file_df['PRJ']=file_df['PRJ'].astype(str)
dird_df['PRJ']=dird_df['PRJ'].astype(str)
final=dird_df.merge(file_df[['PRJ','File','SDG']], on='PRJ',how='left')

In [ ]:
final.head()
final.to_excel('dird_project_summary.xlsx')

In [ ]:
long_str='The idea is to run a loop from start to end and for every index in the given string check whether the sub-string can be formed from that index. This can be done by running a nested loop traversing the given string and in that loop run another loop checking for sub-string from every index. '
short_str='for_every index'

print(short_str in long_str)